In [1]:
title: Expired card reissue: saving on unused cards
author: Vladas Jankus 
date: 2022-11-20
region: EU  
tags: cards, card, expiration, reissue, MCU, product, lapse, MCU categories
summary: Here you will find a data driven suggestion for an algorithm to save money on cards that were reissued but not used. A combination of three criteria could give us a good guess on whether a customer will reactivate new issued card after their old one has expired.

In [2]:
cd /app/

/app


In [3]:
! pip install altair
import altair as alt
import pandas as pd
from utils.datalib_database import df_from_sql
from IPython.display import HTML, display, Markdown as md

# import matplotlib.pyplot as plt

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [4]:
# Create global style for altair charts
def style_theme():
    # Typography
    font = "Karla"
    # At Urban it's the same font for all text but it's good to keep them separate in case you want to change one later.
    labelFont = "Montserrat"
    sourceFont = "Montserrat"
    # Axes
    axisColor = "#000000"
    gridColor = "#DEDDDD"
    # Colors
    markColor = "#88C04D"
    main_palette = [
        "#88C04D",  # green
        "#44A8DA",  # blue
        "#FED234",  # yellow
        "#A989C5",  # purple
        "#066696",  # dblue
        "#FC97C2",  # pink
        "#FF8360",  # orange
        "#B7ABC2",  # purple-gray
    ]
    sequential_palette = [
        "#cfe8f3",
        "#a2d4ec",
        "#73bfe2",
        "#46abdb",
        "#1696d2",
        "#12719e",
    ]
    return {
        # width and height are configured outside the config dict because they are Chart configurations/properties not chart-elements' configurations/properties.
        "width": 800,  # from the guide
        "height": 400,  # not in the guide
        "config": {
            "title": {
                "fontSize": 18,
                "font": font,
                "anchor": "start",  # equivalent of left-aligned.
                "fontColor": "#000000",
            },
            "axisX": {
                "domain": True,
                "domainColor": axisColor,
                "domainWidth": 1,
                "grid": False,
                "labelFont": labelFont,
                "labelFontSize": 12,
                "labelAngle": 0,
                "tickColor": axisColor,
                "tickSize": 5,  # default, including it just to show you can change it
                "titleFont": font,
                "titleFontSize": 12,
                "titlePadding": 10,  # guessing, not specified in styleguide
                "title": "X Axis Title (units)",
            },
            "axisY": {
                "domain": False,
                "grid": True,
                "gridColor": gridColor,
                "gridWidth": 1,
                "labelFont": labelFont,
                "labelFontSize": 12,
                "labelAngle": 0,
                "ticks": False,  # even if you don't have a "domain" you need to turn these off.
                "titleFont": font,
                "titleFontSize": 12,
                "titlePadding": 10,  # guessing, not specified in styleguide
                "title": "Y Axis Title (units)",
                # titles are by default vertical left of axis so we need to hack this
                "titleAngle": 0,  # horizontal
                "titleY": -15,  # move it up
                "titleX": 18,  # move it to the right so it aligns with the labels
            },
            "range": {
                "category": main_palette,
                "diverging": sequential_palette,
            },
            "legend": {
                "labelFont": labelFont,
                "labelFontSize": 12,
                "symbolType": "square",
                "symbolSize": 100,
                "titleFont": font,
                "titleFontSize": 12,
                "title": "",
                "orient": "none",
                "direction": "horizontal",
                "titleAnchor": "middle",
            },
            "view": {
                "stroke": "transparent",  # altair uses gridlines to box the area where the data is visualized. This takes that off.
            },
            "background": {"color": "#D9E9F0"},  # white rather than transparent
            ### MARKS CONFIGURATIONS ###
            "area": {
                "fill": markColor,
            },
            "line": {
                "color": markColor,
                "stroke": markColor,
                "strokeWidth": 2,
            },
            "trail": {
                "color": markColor,
                "stroke": markColor,
                "strokeWidth": 0,
                "size": 1,
            },
            "path": {
                "stroke": markColor,
                "strokeWidth": 0.5,
            },
            "point": {
                "filled": True,
            },
            "text": {
                "font": sourceFont,
                "color": markColor,
                "fontSize": 11,
                "align": "right",
                "fontWeight": 400,
                "size": 11,
            },
            "bar": {
                "size": 40,
                "binSpacing": 1,
                "opacity": 0.8,
                "continuousBandSize": 30,
                "discreteBandSize": 30,
                "fill": markColor,
                "stroke": False,
                "cornerRadiusTopLeft": 3,
                "cornerRadiusTopRight": 3,
            },
            "column": {
                "fontSize": 18,
                "font": font,
                "anchor": "start",  # equivalent of left-aligned.
                "fontColor": "#000000",
            },
        },
    }


alt.themes.register("style_theme", style_theme)
alt.themes.enable("style_theme")

ThemeRegistry.enable('style_theme')

In [5]:
sql_path = "research/product/cards/20220928_card_reorder_activation/queries/"

In [8]:
query = open(sql_path + "mcu_usage_rate.sql", "r").read()
mcu_card_usage = df_from_sql("redshiftreader", query)

In [9]:
query = open(sql_path + "card_abandoned.sql", "r").read()
card_abandoned = df_from_sql("redshiftreader", query)

In [10]:
query = open(sql_path + "mcu_abandoned_cross.sql", "r").read()
mcu_abandoned_cross = df_from_sql("redshiftreader", query)

In [11]:
query = open(sql_path + "reissue_type.sql", "r").read()
reissue_type = df_from_sql("redshiftreader", query)

In [6]:
query = open(sql_path + "pnl_calculation.sql", "r").read()
pnl_calculation = df_from_sql("redshiftreader", query)

In [15]:
query = open(sql_path + "decision.sql", "r").read()
decision = df_from_sql("redshiftreader", query)

# Expired card reissue: saving on unused cards
Vladas Jankus<br/>
2022-11-20

Here you will find a data driven suggestion for an algorithm to save money on cards that were reissued but not used. A combination of three criteria could give us a good guess on whether a customer will reactivate new issued card after their old one has expired. 

# TL;DR

In this research we came up with an algorithm to decide which customers can be proactively approached with an expired card reissue and which customers should not have a proactive approach. All customers must have a possibility to reissue their card, however we can refrain from proactive reissue where we know that the possibility of using reissued cards is low.

In the end, we could increase the new card activation and usage from 50% to 72% by reissuing almost 40% less cards. We don't need to approach customers who have never used any cards (Potential MCUs) and customers who haven't made any card transaction in the past 6 months (Lapsed MCUs). 

All other customers could receive a notification to confirm their shipping address and receive a new card (sync reissue). However if someone does not confirm their shipping address, we should only proactively send out (batch reissue) new cards to customers who are MCUs (Monthly Card Users) and have used the expired card in the past 6 months.

Please read further how we came up with this conclusion.

### Contents:
* [1. Criteria](#section1)
  * [1.1. MCU status](#criterion1)
  * [1.2. Expired card usage](#criterion2)
  * [1.3. Reissue type](#criterion3)
* [2. All criteria applied](#section2)
  * [2.1. Continuous MCUs](#continuous_mcu)
  * [2.2. Sporadic/Undecided MCUs](#sporadic_mcu)
  * [2.3. Dormant MCUs](#dormant_mcu)
  * [2.4. Lapsed MCUs](#lapsed_mcu)
  * [2.5. Potential MCUs](#potential_mcu)
* [3. Algorithm accuracy](#section3)
  * [3.1. Potential activations lost](#pot_activations_lost)
  * [3.2. Potential useless cards reissued](#pot_useless_cards_reissued)
* [4. P&L estimation](#section4)
* [5. Conclusion: possible process](#section5)


# 1. Criteria <a name="section1"></a>
After an initial investigation it was decided to use three criteria for the determination of card reissue.
1. MCU status - information about user card activity in general.
2. Old card last usage - whether the last card was used.
3. Reissue type - whether it was a sync or batch reissue.

A note on possession of additional card criterion. Initially it was assumed that this could change the data significantly, however it does not. It was not found to be an impactful criterion mostly because old card usage is more valuable. In the end it does not matter whether a customer has additional cards or not, the likelihood of reissued card usage depends more strongly on last card usage.

Another note - all cards used in this research are standard. Only standard cards have expired so far, therefore conclusions should be applied with caution for other tiers.

## 1.1. MCU status <a name="criterion1"></a>
MCU status is a good separator of users in terms of their card usage. You can find more about MCU categorisation in [this research](https://research.tech26.de/reports/20220220_mcu_categories.html). However in general we expect different MCU types to behave differently when reactivating their card. See chart below displaying current reissued cards split by their cardholder MCU status (at the time of card reissue).

In [16]:
data = mcu_card_usage
xaxis = "mcu_category"
col_split = "mcu_category"

x_axis_title = "MCU category"
chart_title = "Reissued card usage by MCU status"

base = alt.Chart(data).encode(x=alt.X(xaxis + ":O", axis=alt.Axis(title=x_axis_title)))
bar = base.mark_bar(size=100, opacity=0.3).encode(
    y=alt.Y(
        "customers:Q",
        axis=alt.Axis(title="# of reissued cards", grid=False),
        scale=alt.Scale(domain=[0, 70000]),
    ),
    color=alt.Color(col_split, legend=None),
    order=alt.Order(col_split),
)
bar_activated = base.mark_bar(size=100).encode(
    y=alt.Y("usage_rate:Q", axis=None, scale=alt.Scale(domain=[0, 70000])),
    color=alt.Color(
        col_split,
        legend=alt.Legend(
            title="MCU status", legendX=650, legendY=-15, direction="vertical"
        ),
    ),
    order=alt.Order(col_split),
)
cards_text = bar.mark_text(align="center", size=12, dx=0, dy=-5).encode(
    text=alt.Text("cards_text"),
)
usage_text = bar_activated.mark_text(align="center", size=12, dx=0, dy=-5).encode(
    text=alt.Text("usage_text"),
)

nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=[xaxis], empty="none"
)
selectors = (
    base.mark_point()
    .encode(
        alt.X(xaxis + ":O"),
        opacity=alt.value(0),
    )
    .add_selection(nearest)
)

alt.layer(
    bar_activated + usage_text,
    bar + cards_text,
    selectors
    # text_bar
).properties(title=chart_title).resolve_scale(y="independent").configure(
    background="#fff"
)

alt.LayerChart(...)

What we can see in this chart is that customers active with their cards have over 70% usage rate. While the ones who are not active (dormant, lapsed or potential) on the moment of reissue have significantly lower usage rate. There are several insights that we can make straight away:

1. Potential customers (who have never been MCUs) show no actual potential to activate and use their reissued card. 6% usage rate is very low.
2. Lapsed customers have a low usage rate, however 17% activated still contain a large absolute amount of customers, therefore it would be useful to split these out into a group that has a higher usage rate.
3. Same for Continuous and Sporadic/Undecided customers. There still is 30% of customers who don't use their cards and it makes sense to split these out as well.
4. Dormant customers have a 50% usage rate. From previous research we have information that Dormant customers are either the ones who will return using their card sporadically, or ones who are on their way to being lapsed. Therefore we should also try to split these out. 

In all cases, to split these categories out we will use the further two criteria that we will discuss in this section.

## 1.2. Expired card usage <a name="criterion2"></a>
Second important criterion is the usage of the old expired card. We need to draw a line, on which we can say that the old expired card was abandoned and it is not likely that replacement will be activated. This criterion alone is not a good source of prediction, but it should become very useful when paired with MCU category and possession of secondary card. 

Chart below should help to determine when can we label old card as <b>abandoned</b>. X axis indicates months since last transaction on the old card while lines indicate how many correct predictions we have on each x as well as the overall accuracy. Note: accuracy is the sum of true positive and true negative predictions over all reissued cards.

In [17]:
data = card_abandoned
xaxis = "months_since_last_tx"
yaxis = "sample_size"

x_axis_title = "Months since last transaction"
y_axis_title = "KPI %"
chart_title = (
    "Reissued card activation based on months since last transaction on the old card"
)

text_start = 150
text_gap = 15

base = alt.Chart(data).encode(x=alt.X(xaxis + ":O", axis=alt.Axis(title=x_axis_title)))

bar = base.mark_bar(size=30).encode(
    y=alt.Y(
        "cards:Q",
        axis=None,
    )
)
accuracy = base.mark_line(color="#44A8DA").encode(
    y=alt.Y(
        "accuracy:Q",
        axis=alt.Axis(title=y_axis_title, format=".0%", grid=True),
        scale=alt.Scale(domain=[0, 1]),
    )
)
corr_activations = base.mark_line(color="#FF8360").encode(
    y=alt.Y("perc_positive_detected:Q", axis=None, scale=alt.Scale(domain=[0, 1]))
)
corr_non_activations = base.mark_line(color="#A989C5").encode(
    y=alt.Y("perc_negative_detected:Q", axis=None, scale=alt.Scale(domain=[0, 1]))
)

nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=[xaxis], empty="none"
)
selectors = (
    base.mark_point()
    .encode(
        alt.X(xaxis + ":O"),
        opacity=alt.value(0),
    )
    .add_selection(nearest)
)
ruler = (
    base.mark_rule(color="gray")
    .encode(
        x=xaxis + ":O",
    )
    .transform_filter(nearest)
)

accuracy_text = base.mark_text(align="left", dx=5, size=12, color="#44A8DA").encode(
    y=alt.value(text_start),  # pixels from top
    text=alt.condition(nearest, "accuracy_text:O", alt.value(" ")),
)
accuracy_points = accuracy.mark_point(color="#44A8DA").encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)
corr_activations_text = base.mark_text(
    align="left", dx=5, size=12, color="#FF8360"
).encode(
    y=alt.value(text_start + text_gap),  # pixels from top
    text=alt.condition(nearest, "corr_non_activations_text:O", alt.value(" ")),
)
corr_activations_points = corr_activations.mark_point(color="#FF8360").encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)
corr_non_activations_text = base.mark_text(
    align="left", dx=5, size=12, color="#A989C5"
).encode(
    y=alt.value(text_start + text_gap * 2),  # pixels from top
    text=alt.condition(nearest, "corr_activations_text:O", alt.value(" ")),
)
corr_non_activations_points = corr_non_activations.mark_point(color="#A989C5").encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)


alt.layer(
    bar,
    accuracy,
    corr_activations,
    corr_non_activations,
    selectors,
    ruler,
    accuracy_points,
    accuracy_text,
    corr_activations_text,
    corr_activations_points,
    corr_non_activations_text,
    corr_non_activations_points,
).properties(title=chart_title).resolve_scale(y="independent").configure(
    background="#fff"
)

alt.LayerChart(...)

We can see that the peak accuracy can be achieved at around 4-7 months, where 80% of predictions would be correct. It is reasonable to make a cut at 6 months, which is exactly half-year. 6 months would also match our definition of lapsed MCU who is not likely to return.

Therefore we conclude that a card not used for 6 months is unlikely to be reactivated and used after expiration reissue. 

## 1.3. Reissue type  <a name="criterion3"></a>
Reissue type is not something that a customer has up front, however it gives valuable information on how proactive response to our card reissue notification influences the reissued card usage. Before looking further, let's define the two types of expired card reissue:

1. <b>Sync reissue</b> happens when a customer responds to our notification about their expiring card. Whenever a customer confirms their shipping address and confirms that they want to receive a card replacement, the card is <i>sync reissued</i>
2. <b>Batch reissue</b> happens when the customer does not respond to notification. We can still proactively send the card without a customer response with a <i>batch reissue</i>

In the past card reissues we noticed that activation rate with a batch reissue is significantly lower. This can be seen in the chart below.

In [18]:
data = reissue_type
xaxis = "reissue_type"
x_axis_title = "Reissue type"
chart_title = "Reissued card usage by reissue type"

base = alt.Chart(data).encode(x=alt.X(xaxis + ":O", axis=alt.Axis(title=x_axis_title)))
bar = base.mark_bar(size=100, opacity=0.3).encode(
    y=alt.Y(
        "cards:Q",
        axis=alt.Axis(title="# of reissued cards", grid=False),
        scale=alt.Scale(domain=[0, 180000]),
    ),
    color=alt.Color("reissue_type", legend=None),
)
bar_activated = base.mark_bar(size=100).encode(
    y=alt.Y("used_cards:Q", axis=None, scale=alt.Scale(domain=[0, 180000])),
    color=alt.Color(
        "reissue_type",
        legend=alt.Legend(
            title="Reissue Type", legendX=450, legendY=-15, direction="vertical"
        ),
    ),
)
cards_text = bar.mark_text(align="center", size=12, dx=0, dy=-5).encode(
    text=alt.Text("cards_text"),
)
usage_text = bar_activated.mark_text(align="center", size=12, dx=0, dy=-5).encode(
    text=alt.Text("usage_text"),
)

nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=[xaxis], empty="none"
)
selectors = (
    base.mark_point()
    .encode(
        alt.X(xaxis + ":O"),
        opacity=alt.value(0),
    )
    .add_selection(nearest)
)

alt.layer(
    bar_activated + usage_text,
    bar + cards_text,
    selectors
    # text_bar
).properties(title=chart_title, width=500).resolve_scale(y="independent").configure(
    background="#fff"
)

alt.LayerChart(...)

Essentially it would seem that it is not worth batch reissuing cards to customers because the usage rate after batch reissue is very low. However we can still cross this against other criteria to see that in some cases we can still batch reissue cards. 

# 2. All criteria applied <a name="section2"></a>
In this section we will look at all criteria applied to the cards that were already reissued together. Since we have three dimensions (MCU category, card abandoned, reissue type) it will be easier to discuss the data separated by MCU category. Follow each segment in this section for a different MCU type.

## 2.1. Continuous MCUs <a name="continuous_mcu"></a>
Let's start with continuous MCUs. These are the customers who have been active with their card for over 90% of their lifetime. See chart below for their card reissue activation and usage.

In [19]:
def mcu_category_chart(mcu_group, ydomain):
    data = mcu_abandoned_cross.loc[mcu_abandoned_cross["mcu_category"] == mcu_group]

    xaxis = "category"
    col_split = "category"

    x_axis_title = "Reissue type & expired card status"
    chart_title = mcu_group + " MCUs"

    base = alt.Chart(data).encode(
        x=alt.X(xaxis + ":O", axis=alt.Axis(title=x_axis_title))
    )
    bar = base.mark_bar(size=100, opacity=0.3).encode(
        y=alt.Y(
            "cards:Q",
            axis=alt.Axis(title="# of reissued cards", grid=False),
            scale=alt.Scale(domain=[0, ydomain]),
        ),
        color=alt.Color(col_split, legend=None),
        order=alt.Order(col_split),
    )
    bar_activated = base.mark_bar(size=100).encode(
        y=alt.Y("used_cards:Q", axis=None, scale=alt.Scale(domain=[0, ydomain])),
        color=alt.Color(
            col_split,
            legend=alt.Legend(
                title="Reissue type & expired card status",
                legendX=30,
                legendY=30,
                direction="vertical",
            ),
        ),
        order=alt.Order(col_split),
    )
    cards_text = bar.mark_text(align="center", size=12, dx=0, dy=-5).encode(
        text=alt.Text("cards_text"),
    )
    usage_text = bar_activated.mark_text(align="center", size=12, dx=0, dy=-5).encode(
        text=alt.Text("usage_text"),
    )

    nearest = alt.selection(
        type="single", nearest=True, on="mouseover", fields=[xaxis], empty="none"
    )
    selectors = (
        base.mark_point()
        .encode(
            alt.X(xaxis + ":O"),
            opacity=alt.value(0),
        )
        .add_selection(nearest)
    )

    chart = (
        alt.layer(
            bar_activated + usage_text,
            bar + cards_text,
            selectors
            # text_bar
        )
        .properties(title=chart_title, width=600)
        .resolve_scale(y="independent")
        .configure(background="#fff")
    )

    chart.display()


mcu_category_chart("Continuous", 47000)

alt.LayerChart(...)

For continuous MCUs we definitely have to send a notification for sync reissue. However customers who reply to sync reissue and have their old card abandoned have only 34% of usage rate. I would suggest to still include these customers in the sync reissue process, because the sum number of these customers is quite high and this 34% still makes up with a high number of customers that return using their card (at least once).

## 2.2. Sporadic/Undecided MCUs <a name="sporadic_mcu"></a>
These are the customers that mostly use N26 cards sporadically. They usually move between dormant and active stages with larger gaps of inactivity. See the split below for this type of MCU.

In [20]:
mcu_category_chart("Sporadic/Undecided", 38000)

alt.LayerChart(...)

For Sporadic/Undecided MCUs - only synch reissue where the old card was recently used has a good return to usage ratio. Batch reissue and used cards and sync reissue but abandoned cards return to usage at 39% and 32% respectively. Both are higher than our threshold of 30%. This suggests that we should send notification to all Sporadic/Undecided MCUs and then batch reissue only where the old card was used.

## 2.3. Dormant MCUs <a name="dormant_mcu"></a>
Let's now look at Dormant MCUs who have been inactive for less than 6 months before the card reissue. Usually a part of these customers will return to <b>Sporadic</b> category, and the other part is on their way to <b>Lapsed</b>.

In [21]:
mcu_category_chart("Dormant", 20000)

alt.LayerChart(...)

For dormant MCUs we can see that only sync reissue returns usage rate above the threshold of 30%. This would mean that we can send a notification for dormant MCUs, but don't batch reissue proactively.

## 2.4. Lapsed MCUs <a name="lapsed_mcu"></a>
Lapsed MCUs are customers who have not used their (any) card at least 6 months before the reissue. In this case all cards are abandoned, so you can only see the split between reissue types. See chart below for their reactivation.

In [22]:
mcu_category_chart("Lapsed", 38000)

alt.LayerChart(...)

None of the lapsed MCUs have a high usage rate, therefore it is suggested not to proactively reissue cards to lapsed MCUs at all. Nonetheless it is important to note, that 22% of used cards for sync reissue make up quite a high total number. This potentially could be an exception to the rule, because at the cost of almost 80% unused cards we can return quite some number of customers from lapse. This point would probably require deeper investigation to see if these customers stay active in mid/long-term.

## 2.5. Potential MCUs <a name="potential_mcu"></a>
The last ones are potential MCUs. These customers have never used any card. Same as with lapsed, all their cards are labelled as abandoned and we will only see the split between their reissue type.

In [23]:
mcu_category_chart("Potential", 15000)

alt.LayerChart(...)

Potential MCUs have the lowest usage rates, therefore there is no use of reissuing their cards proactively at all.

# 3. Algorithm accuracy <a name="section3"></a>
In this section we will look at the full impact of our algorithm. We will try to evaluate the accuracy of the algorithm by looking at potentially used cards that we won't be reissuing and potentially useless cards that we will be sending out.

## 3.1 Potential activations lost <a name="pot_activations_lost"></a>
At first let's look at the impact of positive prediction. The chart below displays all <b>activated cards</b> split by the criteria we discussed above. X-axis displays MCU group, while color displays other two criteria - expired card usage and reissue type.

<b>You can see a line at 30%</b>. This is our criteria for reissue, all groups below this criteria would be not reissued. <b>Bubble size</b> corresponds to the percentage of all active and used reissued cards.

In [24]:
data = mcu_abandoned_cross
# xaxis = 'mcu_category'
xaxis = "mcu_category"
x_axis_title = "MCU category"
yaxis = "usage_rate"
y_axis_title = "Usage likelihood"
chart_title = "Active cards split by all criteria"

base = (
    alt.Chart(data)
    .mark_circle()
    .encode(
        x=alt.X(
            xaxis + ":O", axis=alt.Axis(title=x_axis_title), scale=alt.Scale(zero=False)
        )
    )
)

points = base.mark_circle().encode(
    y=alt.Y(
        yaxis + ":Q",
        axis=alt.Axis(title=y_axis_title, format=".0%", grid=True),
        scale=alt.Scale(zero=False, padding=1),
    ),
    color=alt.Color(
        "category", legend=alt.Legend(orient="right", direction="vertical")
    ),
    size=alt.Size("used_cards", scale=alt.Scale(range=[10, 20000]), legend=None),
)

nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=[yaxis], empty="none"
)
selectors = (
    base.mark_point(color="white", size=10)
    .encode(
        y=alt.Y(
            yaxis + ":Q",
            axis=None,
            scale=alt.Scale(zero=False, padding=1),
        ),
        opacity=alt.value(1),
    )
    .add_selection(nearest)
)
point_select = points.mark_circle().encode(
    y=alt.Y(
        yaxis + ":Q",
        axis=None,
        scale=alt.Scale(zero=False, padding=1),
    ),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
)
text_positive_rate = base.mark_text(
    align="center", dx=0, dy=-5, size=12, color="black"
).encode(
    y=alt.Y(yaxis + ":Q", axis=None, scale=alt.Scale(zero=False, padding=1)),
    text=alt.condition(nearest, "used_share_text:O", alt.value(" ")),
)
line = (
    alt.Chart(pd.DataFrame({"y": [0.3]}))
    .mark_rule(strokeDash=[5, 1])
    .encode(y=alt.Y("y", axis=None, scale=alt.Scale(domain=[0, 1])))
)
left_px = 340
top_px = 253
text_above = (
    alt.Chart()
    .mark_text(align="left", baseline="top", color="black", size=25)
    .encode(
        x=alt.value(left_px),  # pixels from left
        y=alt.value(top_px),  # pixels from top
        text=alt.value(["↑ 89% of active cards reissued ↑"]),
    )
)
text_below = (
    alt.Chart()
    .mark_text(align="left", baseline="top", color="black", size=25)
    .encode(
        x=alt.value(left_px),  # pixels from left
        y=alt.value(top_px + 28),  # pixels from top
        text=alt.value(["↓ 11% of active cards lost ↓"]),
    )
)

alt.layer(
    points, selectors, point_select, text_positive_rate, line, text_above, text_below
).properties(title=chart_title).resolve_scale(y="independent").configure(
    background="#fff"
)

alt.LayerChart(...)

We would lose about 10% of all cards that would end up activated and used. By far the largest part of them is concentrated on <b style="color: #A989C5">lapsed MCUs with sync reissue</b>. In the next chart we can see that it also has by far the largest share of inactive cards, thus it is reasonable to leave them behind the line.

We can also note, that there are three <b style="color: #066696"> sync and abandoned </b> clusters that are just right above our threshold. We could consider not reissuing them as well. E.g. if we only exclude Dormant and Sporadic/Undecided MCUs it would add only around 3% more of cards that would be eventually used.

# 3.2. Potentially useless cards reissued <a name="pot_useless_cards_reissued"></a>
In the same manner we can look at cards that would get reissued but never used. The chart below displays exactly the same data, but bubble size now is the share of not-used cards.

In [25]:
data = mcu_abandoned_cross
# xaxis = 'mcu_category'
xaxis = "mcu_category"
x_axis_title = "MCU category"
yaxis = "usage_rate"
y_axis_title = "Usage likelihood"
chart_title = "Not active cards split by all criteria"

base = (
    alt.Chart(data)
    .mark_circle()
    .encode(
        x=alt.X(
            xaxis + ":O", axis=alt.Axis(title=x_axis_title), scale=alt.Scale(zero=False)
        )
    )
)

points = base.mark_circle().encode(
    y=alt.Y(
        yaxis + ":Q",
        axis=alt.Axis(title=y_axis_title, format=".0%", grid=True),
        scale=alt.Scale(zero=False, padding=1),
    ),
    color=alt.Color(
        "category", legend=alt.Legend(orient="right", direction="vertical")
    ),
    size=alt.Size("unused_card_share", scale=alt.Scale(range=[10, 20000]), legend=None),
)

nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=[yaxis], empty="none"
)
selectors = (
    base.mark_point(color="white", size=10)
    .encode(
        y=alt.Y(
            yaxis + ":Q",
            axis=None,
            scale=alt.Scale(zero=False, padding=1),
        ),
        opacity=alt.value(1),
    )
    .add_selection(nearest)
)
point_select = points.mark_circle().encode(
    y=alt.Y(
        yaxis + ":Q",
        axis=None,
        scale=alt.Scale(zero=False, padding=1),
    ),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
)
text_negative_rate = base.mark_text(
    align="center", dx=0, dy=-5, size=12, color="black"
).encode(
    y=alt.Y(yaxis + ":Q", axis=None, scale=alt.Scale(zero=False, padding=1)),
    text=alt.condition(nearest, "unused_share_text:O", alt.value(" ")),
)
line = (
    alt.Chart(pd.DataFrame({"y": [0.3]}))
    .mark_rule(strokeDash=[5, 1])
    .encode(y=alt.Y("y", axis=None, scale=alt.Scale(domain=[0, 1])))
)
left_px = 320
top_px = 253
text_above = (
    alt.Chart()
    .mark_text(align="left", baseline="top", color="black", size=25)
    .encode(
        x=alt.value(left_px),  # pixels from left
        y=alt.value(top_px),  # pixels from top
        text=alt.value(["↑ 35% of not-active cards reissued ↑"]),
    )
)
text_below = (
    alt.Chart()
    .mark_text(align="left", baseline="top", color="black", size=25)
    .encode(
        x=alt.value(left_px),  # pixels from left
        y=alt.value(top_px + 28),  # pixels from top
        text=alt.value(["↓ 65% of not-active cards saved ↓"]),
    )
)


alt.layer(
    points, selectors, point_select, text_negative_rate, line, text_above, text_below
).properties(title=chart_title).resolve_scale(y="independent").configure(
    background="#fff"
)

alt.LayerChart(...)

If we draw the line at 30%, we would save around 65% of cards that will likely end up not used. At the same time we would still reissue 35% of cards that will potentially be not used.

If we would decide to slightly raise the 30% bar and include <b style="color: #066696"> sync and abandoned </b> clusters for Dormant and Sporadic/Undecided MCUs, that would add almost 7% of more cards that potentially won't be used.

It is interesting that by far the largest concentration is in <b style="color: #A989C5"> sync reissue for lapsed and potential MCUs</b>. These customers responded to the notification, but their activation and usage rate is still very low.

# 4. P&L estimation <a name="section4"></a>
In this section we can try to estimate the direct card profit and loss that we would get from the new algorithm. This will include the following parts:

1. MPTS processing cost
2. MPTS issuing cost
3. MDES processing cost
4. MDES token costs
5. Issuer fees

These five costs and fees were summed together and annualised for each card. The table below displays theoretical yearly P&L values per card. Used and unused cards are separated into two columns. In the last column we can see one sum value for all cards in the category.

In [26]:
css_file = "research/product/cards/20220928_card_reorder_activation/css/table_css.txt"
with open(css_file, "r") as table_css:
    style = table_css.read()

pnl = pnl_calculation
pnl = pnl_calculation.rename(
    columns={
        "category": "Category",
        "cards": "Cards",
        "usage_rate": "Cards Used",
        "decision": "Decision",
        "used_first_year_pnl": "P&L / used",
        "not_used_first_year_pnl": "P&L / unused",
        "total_pnl": "P&L sum",
    }
)
pnl.set_index("Category", inplace=True)

pnl = pnl.to_html()
pnl = pnl.replace("dataframe", "customtable")
# pnl
pnl = pnl.replace(
    '<tr style="text-align: right;">\n      <th></th>',
    '<tr style="text-align: left;">\n      <th>Category</th>',
)
pnl = pnl.replace(
    "<th>Category</th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n    </tr>\n",
    "",
)

HTML("""<div>{1}{0}</div>""".format(pnl, style))

Category,Cards,Cards Used,Decision,P&L / used,P&L / unused,P&L sum
Continuous MCU & Sync reissue & card used,47238,89.9%,Reissue,€11.71,€-0.3,€538823
Lapsed MCU & Sync reissue,40793,23.1%,No reissue,-,€-1.68,€-68464
Sporadic/Undecided MCU & Sync reissue & card used,38875,85.1%,Reissue,€7.19,€-0.46,€261371
Lapsed MCU & Batch reissue,21640,3.6%,No reissue,-,€-3.95,€-85560
Dormant MCU & Sync reissue & card used,19015,63.6%,Reissue,€1.21,€-1.07,€2707
Continuous MCU & Sync reissue & card abandoned,17151,34.5%,Reissue,€-0.3,€-1.83,€-36505
Potential MCU & Sync reissue,12622,7.8%,No reissue,-,€-1.67,€-21087
Sporadic/Undecided MCU & Sync reissue & card abandoned,6175,32.4%,Reissue,€-0.27,€-1.88,€-13292
Potential MCU & Batch reissue,6114,1.9%,No reissue,-,€-4.07,€-24899
Dormant MCU & Sync reissue & card abandoned,4934,31.8%,Reissue,€-0.01,€-1.88,€-9352


This topic would probably require a separate research, but what we can notice in the table is that even some used reissued cards have a negative PnL. In all cases where we see a positive PnL per used card, the expired card was used. It looks like this is a good indicator for a used reissued card. However in some categories, e.g. <i>Dormant MCU & Batch reissue & card used</i>, low percentage of used card and low P&L of unused cards overweights the overall sum P&L which ends up negative.

An interesting point is also that in all cases where we had <i>Sync reissue & card abandoned</i> cards end up with a negative sum P&L. From charts above we can see that these categories make up to [17% of all unused cards] and [9% of all used cards]. Interesting is also that even used reissued cards have a negative P&L on all of these categories. However we don't offer to exclude these cards from reissuing because it's only the first-line P&L that cards generate, and we don't know what is the impact of overall P&L for these users. It would require more investigation on these categories to conclude whether it is beneficial to proactively offer a reissue.

# 5 Conclusion - possible process <a name="section5"></a>
There two step-by-step decisions that are needed before reissuing cards:
1. Do we send a notification for sync reissue?
2. Do we do a batch reissue if the customer did not respond to a sync notification?

Looking at the data we have above, we can take MCU category and expired card status and assign actions for batch and sync reissue. The table below writes the reissue decision based on reissued card usage rate. When the rate is below 30% - no reissue. 

Three last columns indicate what would have been the numbers on currently reissued cards with this algorithm.

In [27]:
decision
dec = decision.drop(columns=["sync_used_rate", "batch_used_rate"])
dec = dec.rename(
    columns={
        "category": "Category",
        "usage_rate_before": "Usage rate now",
        "cards_reissued_before": "Cards reissued now",
        "sync_decision": "Sync decision",
        "batch_decision": "Batch decision",
        "usage_rate_after": "Usage rate after",
        "cards_reissued_after": "Cards reissued after",
        "reissued_volume_change": "Reissued volume change",
    }
)
dec.set_index("Category", inplace=True)
dec.drop("rn", axis=1, inplace=True)

dec = dec.to_html()
dec = dec.replace("dataframe", "customtable")
dec
dec = dec.replace(
    '<tr style="text-align: right;">\n      <th></th>',
    '<tr style="text-align: left;">\n      <th>Category</th>',
)
dec = dec.replace(
    "<th>Category</th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n",
    "",
)
dec = dec.replace(
    "<tr>\n      <th>Total</th>",
    '<tr style="background-color:#d2d3d4">\n      <th>Total</th>',
)

HTML("""<div>{1}{0}</div>""".format(dec, style))

Category,Usage rate now,Cards reissued now,Sync decision,Batch decision,Usage rate after,Cards reissued after,Reissued volume change
Continuous & used,89%,48512,Sync reissue,Batch reissue,89%,48512,0%
Sporadic/Undecided & used,83%,40479,Sync reissue,Batch reissue,83%,40479,0%
Dormant & used,59%,21346,Sync reissue,No reissue,64%,19015,-11%
Continuous & abandoned,32%,19771,Sync reissue,No reissue,34%,17151,-13%
Sporadic/Undecided & abandoned,30%,7175,Sync reissue,No reissue,32%,6175,-14%
Dormant & abandoned,27%,6330,Sync reissue,No reissue,32%,4934,-22%
Lapsed,16%,62433,No sync,No reissue,,0,-100%
Potential,6%,18736,No sync,No reissue,,0,-100%
Total,49%,224782,,,72%,136266,-39%


Looking at the Total row, we can see that we could potentially reduce almost 40% of all cards we send out and hope to increase reactivated card usage rates from 50% to 72%. 

This would require not to reissue any cards to Lapsed and Potential MCUs, and batch reissue only to active MCUs where the expired card was used in the past 6 months.

Additionally, we could think about not reissuing cards in all cases where the old card was abandoned. This would increase usage rate even more. However we don't know whether these customers generate anything aside from their card. This point would require further research.